### Раскомментить и установить расширение

In [30]:
# pip install ifcopenshell 


In [2]:
# pip install Pillow

In [3]:
# pip install nbformat

In [4]:
# pip install plotly 

In [5]:
# pip install seaborn

In [6]:
# pip install pandas

# Исполняемый код

In [15]:
import ifcopenshell
import pandas as pd

In [16]:
def extract_ifc_properties_to_dataframe(ifc_path, object_type, properties_list) -> pd.DataFrame: 
    # Загрузка IFC файла
    file = ifcopenshell.open(ifc_path)
    
    # Получение всех объектов заданного типа
    objects = file.by_type(object_type)
    
    # Подготовка данных для DataFrame (pandas)
    data = []

    # print(objects[0].get_info())

    for obj in objects:
        obj_data = {'GlobalId': obj.GlobalId, 'obj_type': obj.get_info()['type']}
        for property_set in obj.IsDefinedBy:
            if property_set.is_a('IfcRelDefinesByProperties'):
                property_set_definition = property_set.RelatingPropertyDefinition
                if property_set_definition.is_a('IfcPropertySet'):
                    for (pset_name, prop_name) in properties_list:
                        if property_set_definition.Name == pset_name:
                            for property in property_set_definition.HasProperties:
                                if property.Name == prop_name and property.is_a('IfcPropertySingleValue'):
                                    # Добавляем значение свойства в данные объекта
                                    if property.NominalValue != None:
                                        obj_data[f'{pset_name}.{prop_name}'] = property.NominalValue.wrappedValue
        data.append(obj_data)

    # Создание DataFrame
    df = pd.DataFrame(data)
    df.set_index('GlobalId', inplace=True)
    
    return df


In [17]:

# Пример использования функции
ifc_path = '12002-81-020601-КМ3.1.3.ifc'
object_type = 'IfcElement'

# Кортеж из интересующих ('propertyset name','property name')
properties_list = [
    ('Tekla Quantity', 'Weight'),
    ('Идентификация', "Категория ТСС"),
    ('Геометрические параметры', 'Высота'),
    ('Геометрические параметры', 'Длина'),
    ('Маркировка', 'Сечение в каталоге Tekla')
]

df = extract_ifc_properties_to_dataframe(ifc_path, object_type, properties_list)
df


{'id': 67, 'type': 'IfcBeam', 'GlobalId': '0cryUIbA55I9w6zXlNGfwV', 'OwnerHistory': #45=IfcOwnerHistory(#3,#4,$,.NOCHANGE.,$,$,$,1706018400), 'Name': 'Балка', 'Description': 'U16P_8240_97', 'ObjectType': 'U16P_8240_97', 'ObjectPlacement': #50=IfcLocalPlacement(#43,#49), 'Representation': #66=IfcProductDefinitionShape($,$,(#64)), 'Tag': 'ID26d7c792-94a1-4548-9e86-f61bd7429e9f'}


,obj_type,Геометрические параметры.Высота,Геометрические параметры.Длина,Маркировка.Сечение в каталоге Tekla,Tekla Quantity.Weight,Идентификация.Категория ТСС
GlobalId,,,,,,
0cryUIbA55I9w6zXlNGfwV,IfcBeam,160.0,29601.9,U16P_8240_97,420.60,NaN
0JG_ZbA9T1Q8XB5M4qoezx,IfcBeam,160.0,2676.0,U16P_8240_97,38.02,NaN
2RTzw4jbf0FQmMYmColiZh,IfcMember,80.0,1735.7,PK80X5.0_32931_2015,19.57,NaN
0D$inzAnnDXvy7fR5jDtv5,IfcBeam,80.0,1150.0,PK80X5.0_32931_2015,12.96,NaN
3_Vu0EUSLDoBPiwg2fsVcI,IfcMember,80.0,1890.1,PK80X5.0_32931_2015,21.31,NaN
...,...,...,...,...,...,...
2JPJHsQLT1zOPsMqvJDdo9,IfcBeam,160.0,4980.0,U16P_8240_97,70.76,Лестницы
3NKQ$V9k1F3gAKixq4dbqM,IfcBeam,300.0,7974.8,I30H2_57837_2017,547.02,Балки
131ujOmhvA8QUlDsviILBl,IfcBeam,200.0,3479.0,U20P_8240_97,63.91,Балки


In [21]:
df.loc[:,'obj_type'].value_counts()

obj_type
IfcBeam                  920
IfcPlate                 750
IfcFastener              522
IfcMember                237
IfcMechanicalFastener    194
IfcColumn                147
Name: count, dtype: int64

In [25]:
df.loc[df['obj_type'] == 'IfcBeam',['obj_type','Маркировка.Сечение в каталоге Tekla']]

,obj_type,Маркировка.Сечение в каталоге Tekla
GlobalId,,
0cryUIbA55I9w6zXlNGfwV,IfcBeam,U16P_8240_97
0JG_ZbA9T1Q8XB5M4qoezx,IfcBeam,U16P_8240_97
0D$inzAnnDXvy7fR5jDtv5,IfcBeam,PK80X5.0_32931_2015
0pkwX3ikD4_efCR4HnegNh,IfcBeam,PK80X5.0_32931_2015
3Ng8LAREv6BABhOPewuZKz,IfcBeam,PK80X5.0_32931_2015
...,...,...
2JPJHsQLT1zOPsMqvJDdo9,IfcBeam,U16P_8240_97
3NKQ$V9k1F3gAKixq4dbqM,IfcBeam,I30H2_57837_2017
131ujOmhvA8QUlDsviILBl,IfcBeam,U20P_8240_97


In [22]:
df.describe()

,Геометрические параметры.Высота,Геометрические параметры.Длина,Tekla Quantity.Weight
count,1304.000000,1304.000000,2054.000000
mean,108.114110,2247.817945,42.984031
std,78.078829,3849.917313,175.127039
min,24.000000,12.000000,0.000000
25%,55.000000,500.000000,1.000000
50%,80.000000,1351.800000,11.280000
75%,160.000000,3149.000000,23.030000
max,800.000000,42740.900000,3200.000000


In [ ]:
# pip install nbformat

  Using cached jsonschema-4.21.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.33.0-py3-none-any.whl.metadata (2.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 857.3 kB/s eta 0:00:000:00:01
Using cached jsonschema-4.21.1-py3-none-any.whl (85 kB)
Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl (18 kB)
Using cached referencing-0.33.0-py3-none-any.whl (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.0/333.0 kB 1.1 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [28]:
df.to_csv('ifc_extracted_data.csv', sep=",")

In [12]:
import plotly.express as px
import seaborn as sns

In [31]:
fig = px.histogram(df, x="Tekla Quantity.Weight", nbins=30)
fig.show()

In [11]:

def data_mean():
sample_data = pd.DataFrame({
    'income': df.sample(1000),
    'type': 'Data',
})

sample_mean_05 = pd.DataFrame({
    'income': [df.sample(5).mean() for _ in range(1000)],
    'type': 'Mean of 5',
})

sample_mean_20 = pd.DataFrame({
    'income': [df.sample(20).mean() for _ in range(1000)],
    'type': 'Mean of 20',
})

results = pd.concat([sample_data, sample_mean_05, sample_mean_20])

,Model,Year,Encoder,Task,Score
0,ERNIE,2019,Transformer,CoLA,75.5
1,T5,2019,Transformer,CoLA,71.6
2,RoBERTa,2019,Transformer,CoLA,67.8
3,BERT,2018,Transformer,CoLA,60.5
4,BiLSTM+ELMo,2018,LSTM,CoLA,32.1
...,...,...,...,...,...
59,BERT,2018,Transformer,RTE,70.1
60,BiLSTM+ELMo,2018,LSTM,RTE,57.4
61,BiLSTM+CoVe,2017,LSTM,RTE,52.7
62,BiLSTM+Attn,2017,LSTM,RTE,58.4
